In [1]:
import requests, json, os, sys, time, re
from datetime import datetime
from multiprocessing import Pool
import sqlalchemy
from sqlalchemy import *
import pymysql

In [2]:
def split_list(lst_long,n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = len(lst_long) // n
    else:
        totalBatches = len(lst_long) // n + 1
    for i in range(totalBatches):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted


def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 1. * currentCount / totalCount * 100
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\rStatus: [{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print('\n')

In [3]:
path_user_id = 'data/steam_user_id.txt'
with open(path_user_id, 'rb') as f:
    lst_user_id = f.readlines()[:5000]

In [4]:
def worker(lst_user_id_temp):
    dic_temp = {}
    for user_id in lst_user_id_temp:
        base_url = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
        params = {
            'key' : '<your personal steam id key>',
            'steamid' : user_id.strip(),
            'format' : 'json' }
        r = requests.get(base_url, params = params)
        user_inventory = r.json().get('response').get('games')
        dic_temp.update({user_id.strip():user_inventory})
        time.sleep(.5)
    return dic_temp

In [5]:
p = Pool(2)

total_count = len(lst_user_id)
current_count = 0
show_work_status(0, total_count, current_count)

dic_master = {}
for i in split_list(lst_user_id,50):
    lst_temp_dic = p.map(worker, split_list(i,25))
    for j in lst_temp_dic:
        dic_master.update(j)
    show_work_status(len(i), total_count, current_count)
    current_count += len(i)
    time.sleep(5)

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [6]:
with open('data/crawled_user_inventory.txt', 'wt') as f:
    for user_id, user_inventory in dic_master.items():
        f.write(json.dumps({user_id.decode("utf-8"):user_inventory}))
        f.write('\n')

### steam app info

In [7]:
r = requests.get('http://steamspy.com/api.php?request=all')
dic_app_user = r.json()

In [10]:
with open('2018-12-16.json', 'w') as f:
    json.dump(r.json(), f, indent = 2)

In [12]:
len(dic_app_user)

27056

In [57]:
path_app_stats = {"owners": {}, "average_2weeks": {}}

In [58]:
for key, value in dic_app_user.items():
    path_app_stats["owners"][key] = value['owners']

In [59]:
for key, value in dic_app_user.items():
    path_app_stats["average_2weeks"][key] = value['average_2weeks']

In [60]:
path_app_user = 'data/path_app_user.json'

with open(path_app_user, 'w') as f:
    json.dump(path_app_stats, f)

### App detail information

In [11]:
path_app_detail = 'data/app_detail.txt'

In [40]:
lst_app_id = dic_app_user.keys()
total_count = len(lst_app_id)
current_count = 0
show_work_status(0, total_count, current_count)

Status: [                                                                                                    ] 0.00% 

In [57]:
with open(path_app_detail, 'wt') as f:
    for app_id in lst_app_id:
        url_app_detail = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id)
        for i in range(3):
            try:
                r = requests.get(url_app_detail)
                result = r.json()
                break
            except:
                time.sleep(1)
                pass
        f.write(json.dumps(result))
        f.write('\n')
        show_work_status(1, total_count, current_count)
        current_count += 1
        if current_count % 200 == 0:
            time.sleep(300)
        else:
            time.sleep(.1)

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [13]:
with open(path_app_detail, 'r') as f:
    dic_steam_app = {'initial_price':{},'name':{},'score':{},'windows':{},'mac':{},'linux':{},'type':{},'release_date':{},'recommendation':{},'header_image':{}}
    lst_raw_string = f.readlines()[:3000]
    total_count = len(lst_raw_string)
    current_count = 0
    show_work_status(0, total_count, current_count)
    for raw_string in lst_raw_string:
        app_data = list(json.loads(raw_string).values())[0].get('data')
        if app_data != {} and app_data:
            steam_id = app_data.get('steam_appid')
            initial_price = app_data.get('price_overview',{}).get('initial')
            if app_data.get('is_free') == True:
                initial_price = 0
            app_name = app_data.get('name')
            critic_score = app_data.get('metacritic', {}).get('score')
            app_type = app_data.get('type')
            for (platform, is_supported) in app_data.get('platforms',{}).items():
                if is_supported == True:
                    dic_steam_app[platform].update({steam_id:1})
            if app_data.get('release_date',{}).get('coming_soon') == False:
                release_date = app_data.get('release_date',{}).get('date')
                if not release_date == '':
                    try:
                        release_date = datetime.strptime(release_date, '%b %d, %Y')
                    except:
                        try:
                            release_date = datetime.strptime(release_date, '%d %b, %Y')
                        except:
                            release_date = None
            # release_date = 'coming_soon'                
            else:
                release_date = 'coming_soon'
                
                    

            recommendation = app_data.get('recommendations',{}).get('total')
            header_image = app_data.get('header_image')
            dic_steam_app['initial_price'].update({steam_id:initial_price})
            dic_steam_app['name'].update({steam_id:app_name})
            dic_steam_app['score'].update({steam_id:critic_score})
            dic_steam_app['type'].update({steam_id:app_type})
            dic_steam_app['release_date'].update({steam_id:release_date})
            dic_steam_app['recommendation'].update({steam_id:recommendation})
            dic_steam_app['header_image'].update({steam_id:header_image})
        show_work_status(1, total_count, current_count)
        current_count += 1

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [14]:
import pandas as pd
df_app_info = pd.DataFrame(dic_steam_app)

In [15]:
df_app_info.head()

,initial_price,name,score,windows,mac,linux,type,release_date,recommendation,header_image
10,999.0,Counter-Strike,88.0,1,1.0,1.0,game,2000-11-01,62533.0,https://steamcdn-a.akamaihd.net/steam/apps/10/...
20,499.0,Team Fortress Classic,NaN,1,1.0,1.0,game,1999-04-01,2656.0,https://steamcdn-a.akamaihd.net/steam/apps/20/...
30,499.0,Day of Defeat,79.0,1,1.0,1.0,game,2003-05-01,1902.0,https://steamcdn-a.akamaihd.net/steam/apps/30/...
40,499.0,Deathmatch Classic,NaN,1,1.0,1.0,game,2001-06-01,888.0,https://steamcdn-a.akamaihd.net/steam/apps/40/...
50,499.0,Half-Life: Opposing Force,NaN,1,1.0,1.0,game,1999-11-01,4102.0,https://steamcdn-a.akamaihd.net/steam/apps/50/...


In [16]:
df_app_info.shape

(2983, 10)

In [18]:
engine = create_engine('mysql+pymysql://root:<password>@localhost:3306/game_recommendation?charset=utf8mb4') #charset=utf8mb4 
#root:password@localhost:3306/database_name’)
df_app_info.to_sql('tbl_app_info_test',engine, if_exists='replace')

InternalError: (pymysql.err.InternalError) (1366, "Incorrect string value: '\\xE3\\x82\\xB9\\xE3\\x83\\x88...' for column 'name' at row 987") [SQL: 'INSERT INTO tbl_app_info_test (`index`, initial_price, name, score, windows, mac, linux, type, release_date, recommendation, header_image) VALUES (%(index)s, %(initial_price)s, %(name)s, %(score)s, %(windows)s, %(mac)s, %(linux)s, %(type)s, %(release_date)s, %(recommendation)s, %(header_image)s)'] [parameters: ({'index': 10, 'initial_price': 999.0, 'name': 'Counter-Strike', 'score': 88.0, 'windows': 1, 'mac': 1.0, 'linux': 1.0, 'type': 'game', 'release_date': datetime.datetime(2000, 11, 1, 0, 0), 'recommendation': 62533.0, 'header_image': 'https://steamcdn-a.akamaihd.net/steam/apps/10/header.jpg?t=1528733245'}, {'index': 20, 'initial_price': 499.0, 'name': 'Team Fortress Classic', 'score': None, 'windows': 1, 'mac': 1.0, 'linux': 1.0, 'type': 'game', 'release_date': datetime.datetime(1999, 4, 1, 0, 0), 'recommendation': 2656.0, 'header_ima